REPLACES *CofA_Nightly_Node_FINALE.py*

---

## *copies code from:* 

## `CofA_email_node_catchup.ipynb`

### [click here](http://localhost:8888/notebooks/OneDrive%20-%20Agilent%20Technologies/Documents/_staging/Data-Developer-Droolkit/AGILENT/%5B2020_01_23%5D_CofA_Email_Node_catchup_date_CHECK.ipynb)

## *and:*

## `CofA_G_DRIVE_File_Convention_Check`

### [click here](http://localhost:8888/notebooks/OneDrive%20-%20Agilent%20Technologies/Documents/_staging/Data-Developer-Droolkit/AGILENT/CofA/%5B2020_01_10%5D_CofA_G_DRIVE_file_convention_check.ipynb)

---

~~1) gets current date via `pandas.Timestamp.now()` (will be at `5 am every morning`)~~

~~2) creates date range of `yesterday mornging at 00:00:00` to `current morning at 00:00:00`~~

3) `a)` attempts to read PDF file and determine `created date`

**OTHERWISE:**

3) `b)` scans the `date-modified` of files in the `directories`

4) returns **ONLY** the files created in above date-range

5) copies/watermarks/moves all files to `G:/C of A's/#Email Node/` **AND** `temp folder`

6) creates listing in .csv with `filename` and `timestamp` and .zip of files in temp folder

7) then sends both .csv & .zip files to `agilent_cofa@agilent.com` and CC myself

---

In [3]:
import os, sys, time
from time import sleep
from pathlib import Path
import fnmatch, glob, shutil

In [2]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [4]:
import logging
import tempfile, subprocess
from PyPDF2 import PdfFileReader, PdfFileWriter
from zipfile import ZipFile
import email, smtplib, ssl
import os.path as op
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate

In [4]:
def setup_logger(): # {
    # TRY THE FOLLOWING
    try: # {
        logging.basicConfig(level=logging.INFO,
                           format='%(asctime)s: %(message)s, FROM=%(funcName)s, LINENO=%(lineno)d',
                           datefmt='%Y-%m-%d-%H%M%S',
                           filemode='a')
    # }
    except: #{ 
        errorMessage = str(sys.exc_info()[0]) + "\n"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage))
        print("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
    # }
    else: # {
        logging.info("Operation Completed Successfully...")
    # }
# }

In [5]:
"""
TAKES IN:
(1) file_path to file
RETURNS:
The create time or modified time, whichever is older
"""
def pull_creation_timestamp(a_file_path): # {
    # TRY THE FOLLOWING:
    try: #{
        # FORCE PATH VARIABLE
        the_path = Path(a_file_path)
        # GET MODIFIED TIME
        mtime = os.path.getmtime(the_path)
        # GET CREATE TIME
        ctime = os.path.getctime(the_path)
        # CREATE DATE VAR
        # IF CREATE TIME IS OLDER...
        if ctime < mtime:  # {
            # FORMAT DATE VAR as str
            date_time = datetime.fromtimestamp(ctime)
        # }
        # ELSE.... MODIFIED TIME IS OLDER...
        else:  # {
            # FORMAT DATE VAR as str
            date_time = datetime.fromtimestamp(mtime)
        # }
    #}
    except: #{
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        logging.error("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
    #}
    else:  # {
        logging.info("SUCCESS! VERY NICE!")
        # RETURN THE DATE WE PULLED AS STRING
        return date_time
    #}
    finally:  # {
        logging.info("[pull_creation_timestamp] FIN...")
    #}
# }

In [6]:
"""
TAKES IN: 
(1) path to pdf
RETURNS:
The STRING of that PDF to match naming convention SWITCH
IE:

F:/APPS/CofA/ >>> G:/C of A's/Agilent/ 

G:/C of A's/Agilent/ >>> F:/APPS/CofA/
"""
def generate_naming_convention(the_pdf_path): # {
    ##########################
    # RE-INSTANTIATE GLOBALS #
    global f_drive_convention, g_drive_convention
    logging.info("\n\tthe_pdf_path == " + str(the_pdf_path))
    # TRY THE FOLLOWING
    try: # {
        # GET / SET FILENAME TO VARIABLE
        the_file_name = str(os.path.basename(the_pdf_path))
        # GET /SET DIRNAME to VARIABLE
        the_dir_name = os.path.dirname(the_pdf_path)
        # CHECK FILE NAME CONVENTION (F_DRIVE?)
        if fnmatch.fnmatch(the_file_name, f_drive_convention): # {
            # MATCHES F_DRIVE FILE NAME CONVENTION... "xxxx@xxxx.pdf"
            idx_mrk = the_file_name.rfind('@', 0, len(the_file_name))
            half1 = str(the_file_name[0:idx_mrk])
            half2 = str(the_file_name[idx_mrk + 1:len(the_file_name)])
            print("\n\tHALF 1 == " + half1)
            print(len(half1))
            print("\n\tHALF 2 == " + half2)
            print(len(half2))
            # SETUP NEW FILE NAME (for copy)
            new_name = "part "
            new_name += str(half1)
            new_name += " CofA Lot# "
            new_name += str(half2)
            print("\n\tNEW NAME == " + str(new_name))
        # }
        # ELSE IF... IT MATCHES OTHER NAMING CONVENTION
        elif fnmatch.fnmatch(the_file_name, g_drive_convention): # {
            # MATCHES G_DRIVE FILE NAME CONVENTION... "part xxxx CofA Lot # xxx.pdf"
            idx_mrk = the_file_name.rfind('Lot', 0, len(the_file_name))
            half1 = str(the_file_name[5:idx_mrk-6])
            half2 = str(the_file_name[idx_mrk + 6:len(the_file_name)])
            print("\n\tHALF 1 == '" + half1 + "'")
            print(len(half1))
            print("\n\tHALF 2 == '" + half2 + "'")
            print(len(half2))
            # SETUP NEW FILE NAME (for copy)
            new_name = str(half1)
            new_name += "@"
            new_name += str(half2)
            # REMOVE WHITESPACE FROM STR
            new_name = new_name.rstrip('')
            print("\n\tNEW NAME == " + str(new_name))
        # }
    # }
    except: # {
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        logging.error("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
    # }
    else: # {
        print("\t\t'" + str(new_name) + "'")
        print("\n\tLENGTH OF == " + str(len(new_name)))
    # }
# }

In [7]:
f_drive_convention = "*[@]*.pdf" # REGEX for F_DRIVE
g_drive_convention = "[part]*[CofA]*[Lot]*[#]*.pdf" # REGEX for G_DRIVE

In [8]:
generate_naming_convention(
    the_pdf_path="G:/C of A's/Agilent/5190-6901/part 5190-6901 CofA Lot # CT-0676.pdf"
)


	HALF 1 == '5190-6901'
9

	HALF 2 == 'CT-0676.pdf'
11

	NEW NAME == 5190-6901@CT-0676.pdf
		'5190-6901@CT-0676.pdf'

	LENGTH OF == 21


In [9]:
generate_naming_convention(
    the_pdf_path="F:/APPS/CofA/625-MA-1@0006502723.pdf"
)


	HALF 1 == 625-MA-1
8

	HALF 2 == 0006502723.pdf
14

	NEW NAME == part 625-MA-1 CofA Lot# 0006502723.pdf
		'part 625-MA-1 CofA Lot# 0006502723.pdf'

	LENGTH OF == 38


In [10]:
def create_watermark(input_pdf, output, watermark): # {
    # TRY THE FOLLOWING
    try:  # {
        watermark_obj = PdfFileReader(watermark)
        watermark_page = watermark_obj.getPage(0)

        pdf_reader = PdfFileReader(input_pdf)
        pdf_writer = PdfFileWriter()

        # Watermark all the pages
        for page in range(pdf_reader.getNumPages()):  # {
            page = pdf_reader.getPage(page)
            page.mergePage(watermark_page)
            pdf_writer.addPage(page)
        # }

        with open(output, 'wb') as out:  # {
            pdf_writer.write(out)
        # }
    # }
    except: # {
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        logging.error("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
        # }
    else: # {
        logging.info("\t\t[watermark-pdf] FIN...")
    # }
    finally: # {
        # CREATE END-TIME VAR
        time_end = pd.Timestamp.now()
        # DETERMINE OVERALL RUN-TIME
        run_time = pd.Timedelta(time_end - time_start)
        # PRINT TOTAL RUNTIME
        logging.info("\t\t[watermark-pdf] >>> time_alloted: " + str(run_time))
    # }
    return
# }

In [11]:
def get_all_file_paths(directory): # {
    # TRY THE FOLLOWING
    try: # {
        # initializing empty file paths list
        file_paths = []
        
        # crawling through directory and subdirectories
        for root, directories, files in os.walk(directory):  # {
            for filename in files:  # {
                # join the two strings in order to form the full filepath.
                filepath = os.path.join(root, filename)
                file_paths.append(filepath)
            # }
        # }
        
        # returning all file paths
        return file_paths
    # }
    except: # {
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        logging.error("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
    # }
    else: # {
        logging.info("Operation Completed Successfully...")
    # }
# }

In [12]:
def zip_the_directory(directory_to_zip): # {
    ##########################
    # RE-INSTANTIATE GLOBALS #
    global og_wd
    # TRY THE FOLLOWING
    try: # {
        logging.info("\n\tWORKING DIRECTORY ---BEFORE--- ZIP == " + str(og_wd))
        # CHANGE WORKING DIRECTORY TO DIRECTORY WE WISH TO ZIP
        os.chdir(directory_to_zip)
        # path to folder (NOW currently folder we are in/ want to zip)
        directory = "."
        logging.info("\n\tWORKING DIRECTORY ---DURING--- ZIP == " + str(os.getcwd()))
        # calling function to get all file paths in the directory
        file_paths = get_all_file_paths(directory)
        # printing the list of all files to be zipped
        logging.info("\nFollowing files will be zipped:")
        for file_name in file_paths: # {
            logging.info(file_name)
        # }
        # writing files to a zipfile
        with ZipFile('CofA-'
                    + str(pd.Timestamp.now())[:10]
                    + ".zip",'w') as zip: # {
            # writing each file one by one
            for file in file_paths: # {
                zip.write(file)
            # }
        # }
        logging.info("ALL FILES ZIPPED SUCCESSFULLY!")
    # }
    except: # {
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        logging.error("\n" + typeE +
                      "\n" + fileE +
                      "\n" + lineE +
                      "\n" + messageE)
    # }
    else: # {
        logging.info("Operation Completed Successfully...")
    # }
# }

In [13]:
"""
TAKES IN:
(1) directory to traverse and scan thru all files
(2) list of sub-directories to ignore within (1)
(3) the list of file types to look for
(4) the STARTING DATE timestamp of when files fall under
(5) the ENDING DATE timestamp of when files fall under
(6) the list of 
"""
def dir_traverse(the_directory, ignore_dir_list, file_type_list, check_start, check_end): # {
    ##########################
    # RE-INSTANTIATE GLOBALS #
    global file_list_f, time_list_f
    global file_list_g, time_list_g
    # TRY THE FOLLOWING
    try: # {
        logging.info("\nTS_LIST_F:\t" + str(file_list_f))
        # COUNTER
        x = 0 # number of PDFs matched
        count = 0 # total number of files
        # SETUP VARIABLES
        scan_directory = Path(the_directory)
        # BEGIN OS.WALK
        for root, dirs, files in os.walk(scan_directory): # {
            # FOR EACH ITEM IN IGNORE LIST...
            for item in ignore_dir_list: # {
                # IF THAT ITEM IS IN FACT IN DIRECTORY...
                if str(item) in dirs: # {
                    # REMOVE FROM OS.WALK
                    dirs.remove(str(item))
                # }
            #}
            # OTHERWISE IF THERE ARE FILES IN DIRECTORY
            for f in files: # {
                # FOR EACH ITEM IN 'file_type_list'
                for item in file_type_list: # {
                    # CREATE FILE_MATCH VAR
                    file_match = str("*" + item)
                    # DO FNMATCH FOR THIS "item"
                    if fnmatch.fnmatch(f, file_match): # {
                        # <<< STR OPERATIONS >>>
                        # ASSEMBLE!
                        file_path = os.path.join(root, f)
                        logging.info("\n\tFILE_PATH == " + str(file_path))
                        # RETURN DATETIME VAR
                        the_date = pull_creation_timestamp(file_path)
                        logging.info("\n\tTHE_DATE == " + str(the_date))
                        # CONVERT TO pandas.Timestamp
                        timestamp = pd.Timestamp(the_date)
                        # COMPARE TO CHECK_DATE VARIABLE
                        check_delta = pd.Timestamp(the_date)
                    # }
                # }
            # }
        # }
    # }
    except: # {
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        logging.error("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
    # }
    else: # {
        pass
    # }
# }

In [14]:
def send_email(send_from, send_to, subject, message, files=[],
              server="cos.smtp.agilent.com", port=587, use_tls=True): # {
    # TRY THE FOLLOWING
    try: # {
        print("SENDING MAIL... DATE == " + str(pd.Timestamp.now())[:10])
        msg = MIMEMultipart()
        msg['From'] = send_from
        msg['To'] = COMMASPACE.join(send_to)
        msg['Date'] = formatdate(localtime=True)
        msg['Subject'] = subject
        
        msg.attach(MIMEText(message))
        
        for path in files:  # {
            part = MIMEBase('application', "octet-stream")
            with open(path, 'rb') as file:  # {
                part.set_payload(file.read())
            # }
            encoders.encode_base64(part)
            part.add_header('Content-Disposition',
                            'attachment; filename="{}"'.format(op.basename(path)))
            msg.attach(part)
        # }
        
        smtp = smtplib.SMTP(server, port)
        if use_tls:  # {
            smtp.starttls()
        # }
        smtp.sendmail(send_from, send_to, msg.as_string())
        smtp.quit()
    # }
    except: # {
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        logging.error("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
    # }
    else: # {
        logging.info("Operation Completed Successfully...")
    # }
# }

In [15]:
def main(): # {
    ##########################
    # RE-INSTANTIATE GLOBALS #
    global in_directory_1, in_directory_2, out_directory 
    global ts_now, ts_start_delta, ts_end_delta
    global watermark, og_wd
    ##########################
    logging.info("\n\tTEST-TIME-NOW: " + str(ts_now))
    # SUBTRACT and determine "start_time"
    time_start = ts_now - ts_start_delta
    logging.info("\n\tBEGINING OF SCAN-TIME: " + str(time_start))
    # SUBTRACT and determine "end_time"
    time_end = ts_now - ts_end_delta
    logging.info("\n\tEND OF SCAN-TIME: " + str(time_end))
    # LIST OF DIRECTORIES TO IGNORE IN SCAN
    ignore_list_f = ['Archive ERR',
                   'Archive - For all archived CofA, see G CofA folder',
                   'Instruction Sheets',
                   'EXPORT ERRORS']
    ignore_list_g = ['#Archive']
    # [2020-01-27]\\file_list_f = [] # LIST TO HOLD ALL PATHS FOR FILES (F_DRIVE)
    # [2020-01-27]\\file_list_g = [] # LIST TO HOLD ALL PATHS FOR FILES (G_DRIVE)
    # [2020-01-27]\\ts_list_f = [] # LIST TO HOLD ALL TIMESTAMPS (F_DRIVE)
    # [2020-01-27]\\ts_list_g = [] # LIST TO HOLD ALL TIMESTAMPS (G_DRIVE)
    # SCAN FIRST DIRECTORY
    dir_traverse(the_directory=in_directory_1,
                 ignore_dir_list=ignore_list_f,
                 file_type_list=[".pdf"],
                 check_start=time_start,
                 check_end=time_end
                )
    # SCAN SECOND DIRECTORY
    dir_traverse(the_directory=in_directory_2,
                ignore_dir_list=ignore_list_g,
                 file_type_list=[".pdf"],
                 check_start=time_start,
                 check_end=time_end
                )
    # CREATE DATAFRAMES OF WHICH COFAS ARE NEW AND THEIR TIMESTAMPS
    df_export_f = pd.DataFrame(data=None, columns=None, dtype=np.str)
    # [2020-01-27]\\df_export_f['CofA'] = 
# }

In [16]:
# MAIN BOILERPLATE
if __name__ == "__main__": # {
    # SETUP LOGGER
    setup_logger()
    #############################
    # INSTANTE GLOBAL VARIABLES #
    in_directory_1 = "F:/APPS/CofA/"
    in_directory_2 = "G:/C of A's/Agilent/"
    out_directory = "G:/C of A's/#Email Node/"
    watermark = "C:/data/inbound/Agilent_CofA_Letterhead_03-21-19.pdf"
    og_wd = os.getcwd() # GET/SET ORIGINAL WORKING DIRECTORY
    file_list_f = [] # LIST TO HOLD ALL PATHS FOR FILES (F_DRIVE)
    time_list_f = [] # LIST TO HOLD ALL TIMESTAMPS (F_DRIVE)
    file_list_g = [] # LIST TO HOLD ALL PATHS FOR FILES (G_DRIVE)
    time_list_g = [] # LIST TO HOLD ALL TIMESTAMPS (G_DRIVE)
    f_drive_convention = "*[@]*.pdf" # REGEX for F_DRIVE
    g_drive_convention = "[part]*[CofA]*[Lot]*[#]*.pdf" # REGEX for G_DRIVE
    # CREATE TIMESTAMP FOR RIGHT NOW
    ts_now = pd.Timestamp(year=2020, day=27, month=1, hour=5) # hard-coded for testing
    # USED TO DETERMINE THE CORRECT "start_time" (one day and five hours prior)
    ts_start_delta = pd.Timedelta(days=1, hours=5)
    # USED TO DETERMINE THE CORRECT "end_time" (five hours prior)
    ts_end_delta = pd.Timedelta(hours=5)
    #############################
    # CALL MAIN FUNCTION
    main()
# }

In [17]:
setup_logger()

In [7]:
test_str = str(pd.Timestamp.now)
test_str = str(test_str)[:18]
print(test_str)

<bound method Time
